In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE170999"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE170999"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE170999.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE170999.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE170999.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the Series_summary information, this dataset contains gene expression data from Affymetrix U133 platform
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identifying rows containing trait, age, and gender information
trait_row = 0  # KRAS mutation status is in row 0
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert KRAS mutation status to binary (0: wild-type, 1: mutant)"""
    if value is None:
        return None
    
    # Extract value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if "wild-type" in value.lower():
        return 0  # KRAS wild-type
    elif "mutant" in value.lower():
        return 1  # KRAS mutant
    else:
        return None  # Unknown or other values

def convert_age(value):
    """Convert age to numeric (continuous) value"""
    # Not used since age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary (0: female, 1: male)"""
    # Not used since gender data is not available
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, clinical data is available
if trait_row is not None:
    # Create clinical data DataFrame from the sample characteristics dictionary
    sample_chars = {0: ['kras_mutant or wild_type: KRAS-wild-type', 'kras_mutant or wild_type: KRAS-mutant']}
    clinical_data = pd.DataFrame()
    
    for key, values in sample_chars.items():
        clinical_data[key] = values
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("Clinical Features Preview:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers are not standard human gene symbols, which typically follow conventions like 
# HGNC symbols (e.g., BRCA1, TP53) or Ensembl IDs (starting with ENSG).
# 
# The identifiers shown (like '1007_s_at', '1053_at') appear to be Affymetrix probe IDs,
# which are specific to the microarray platform used for gene expression profiling.
# These need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Based on the previews, we need to map:
# - 'ID' in the gene_annotation dataframe (contains probe IDs like '1007_s_at')
# - 'Gene Symbol' in the gene_annotation dataframe (contains gene symbols like 'DDR1 /// MIR4640')

# 2. Get gene mapping dataframe by extracting the identifier and symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This function handles dividing expression values among multiple genes and summing by gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print preview of gene expression data after mapping
print("Gene expression data after mapping (first 5 genes):")
print(gene_data.head())


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Extract clinical features from the original clinical_data
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row, 
    convert_age=convert_age,
    gender_row=gender_row, 
    convert_gender=convert_gender
)

# Save the clinical features data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Ensure clinical data has the proper format
clinical_df = clinical_features.copy()
clinical_df = clinical_df.T
clinical_df.columns = [trait]

linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
print(f"Linked data shape before handling missing values: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data_cleaned.shape}")

# Check if we have sufficient data after handling missing values
if linked_data_cleaned.shape[0] == 0 or linked_data_cleaned.shape[1] <= 1:
    print(f"Insufficient data after handling missing values. All samples were filtered out.")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Set to False since we have no usable trait data
        is_biased=None, 
        df=linked_data_cleaned,
        note=f"No usable samples after handling missing values. All samples had missing trait values."
    )
    print(f"Data quality check failed. The dataset is not suitable for association studies.")
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

    # 5. Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note=f"Dataset contains gene expression data from rectal cancer patients with KRAS mutation status."
    )

    # 6. Save the data if it's usable
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        # Save the data
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Data quality check failed. The dataset is not suitable for association studies.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract KRAS mutation information from the SOFT file
# Read the SOFT file to look for sample characteristics that indicate KRAS status for each sample
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Extract sample blocks from the SOFT file
sample_blocks = re.findall(r'^\^SAMPLE = (GSM\d+).*?!Sample_title = "(.*?)".*?!Sample_characteristics_ch1 = (.*?)(?=\n\n|\n\^|\Z)', 
                          soft_content, re.DOTALL | re.MULTILINE)

# Create a dictionary to map sample IDs to KRAS status
kras_status = {}
for sample_id, title, characteristics in sample_blocks:
    # Look for KRAS status in the characteristics
    if 'KRAS-mutant' in characteristics or 'KRAS-mutant' in title:
        kras_status[sample_id] = 1  # Mutant
    elif 'KRAS-wild-type' in characteristics or 'KRAS-wild-type' in title:
        kras_status[sample_id] = 0  # Wild-type
    else:
        # If not found in characteristics, try to extract from sample title
        if 'KRAS-mutant' in title.lower():
            kras_status[sample_id] = 1
        elif 'KRAS-wild-type' in title.lower() or 'KRAS-wt' in title.lower():
            kras_status[sample_id] = 0

# Create a clinical DataFrame with sample IDs as index
sample_ids = normalized_gene_data.columns
clinical_df = pd.DataFrame(index=sample_ids)

# Fill in the KRAS status for each sample
clinical_df[trait] = clinical_df.index.map(lambda x: kras_status.get(x))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Sample KRAS status: {clinical_df[trait].value_counts().to_dict()}")

# 3. Link the clinical and genetic data
linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data_cleaned.shape}")

# Check if we still have data after handling missing values
if linked_data_cleaned.shape[0] == 0 or linked_data_cleaned.shape[1] <= 1:
    print("All samples were filtered out during missing value handling.")
    # Create a minimal DataFrame for validation purposes
    dummy_df = pd.DataFrame({trait: [0, 1]})
    # Validate and save information indicating the dataset is not usable
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=True,
        df=dummy_df,
        note="Dataset contains gene expression data from rectal cancer patients with KRAS mutation status, but sample IDs couldn't be properly linked between clinical and genetic data."
    )
    print(f"Data quality check failed. The dataset is not suitable for association studies.")
else:
    # 5. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

    # 6. Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note=f"Dataset contains gene expression data from rectal cancer patients with KRAS mutation status."
    )

    # 7. Save the data if it's usable
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        # Save the data
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Data quality check failed. The dataset is not suitable for association studies.")